<a href="https://colab.research.google.com/github/Panavath/count/blob/main/Train_Count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# tochivsion, this is another library from pytorch we will try to use this method if the yolo does not work well
import torchvision.transforms as transforms
from torchvision.datasets import Food101
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Download Food-101 dataset
train_data = Food101(root='/content/data', split='train', download=True)
test_data = Food101(root='/content/data', split='test', download=True)


# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

print(f"Training dataset size: {len(train_data)} images")
print(f"Test dataset size: {len(test_data)} images")



Training dataset size: 75750 images
Test dataset size: 25250 images


In [ ]:
import os
import random
import shutil

# Define correct paths
dataset_path = "/content/data/food-101/images"  # Corrected path for Google Colab
output_path = "/content/Food101_YOLO"

# Create YOLOv8 folders
os.makedirs(f"{output_path}/images/train", exist_ok=True)
os.makedirs(f"{output_path}/images/val", exist_ok=True)
os.makedirs(f"{output_path}/images/test", exist_ok=True)
os.makedirs(f"{output_path}/labels/train", exist_ok=True)
os.makedirs(f"{output_path}/labels/val", exist_ok=True)
os.makedirs(f"{output_path}/labels/test", exist_ok=True)

# Ensure dataset exists
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset not found at {dataset_path}. Check if it was downloaded correctly.")

# List all category folders
categories = sorted(os.listdir(dataset_path))

# Create a dictionary mapping class names to numeric labels
class_map = {category: idx for idx, category in enumerate(categories)}

# Collect all images and their labels
all_images = []
image_label_map = {}

for category in categories:
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.endswith(".jpg"):
                full_img_path = os.path.join(category_path, img_file)
                all_images.append(full_img_path)
                image_label_map[full_img_path] = class_map[category]  # Assign label

# Shuffle dataset
random.shuffle(all_images)

# Split dataset (80% train, 10% val, 10% test)
train_split = int(0.8 * len(all_images))
val_split = int(0.9 * len(all_images))

train_images = all_images[:train_split]
val_images = all_images[train_split:val_split]
test_images = all_images[val_split:]

# Function to create YOLO labels
def create_yolo_label(image_path, label, save_path):
    """
    Creates a YOLO format label file for an image.
    Since Food-101 is a classification dataset, we assume the whole image as the object.
    YOLO format: class_id x_center y_center width height
    """
    yolo_label = f"{label} 0.5 0.5 1.0 1.0\n"
    label_filename = os.path.join(save_path, os.path.basename(image_path).replace(".jpg", ".txt"))

    with open(label_filename, "w") as f:
        f.write(yolo_label)

# Move images and generate labels
for img in train_images:
    shutil.copy(img, f"{output_path}/images/train/{os.path.basename(img)}")
    create_yolo_label(img, image_label_map[img], f"{output_path}/labels/train")

for img in val_images:
    shutil.copy(img, f"{output_path}/images/val/{os.path.basename(img)}")
    create_yolo_label(img, image_label_map[img], f"{output_path}/labels/val")

for img in test_images:
    shutil.copy(img, f"{output_path}/images/test/{os.path.basename(img)}")
    create_yolo_label(img, image_label_map[img], f"{output_path}/labels/test")

print("✅ Dataset successfully split and labeled in YOLO format!")


✅ Dataset successfully split and labeled in YOLO format!


In [ ]:
import os

dataset_path = "/content/data/food-101/images"

# Count all images in subfolders
image_count = sum([len(files) for _, _, files in os.walk(dataset_path)])

print(f"✅ Total images in dataset: {image_count}")


 Total images in dataset: 101000


In [ ]:
yaml_content = """\
train: /content/Food101_YOLO/images/train
val: /content/Food101_YOLO/images/val
test: /content/Food101_YOLO/images/test

nc: 101  # Number of classes
names: [
    'apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare',
    'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito',
    'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake',
    'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla',
    'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder',
    'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes',
    'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict',
    'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras',
    'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari',
    'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad',
    'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza',
    'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros',
    'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich',
    'macaroni_and_cheese', 'macarons', 'miso_soup', 'mussels', 'nachos',
    'omelette', 'onion_rings', 'oysters', 'pad_thai', 'paella', 'pancakes',
    'panna_cotta', 'peking_duck', 'pho', 'pizza', 'pork_chop', 'poutine',
    'prime_rib', 'pulled_pork_sandwich', 'ramen', 'ravioli', 'red_velvet_cake',
    'risotto', 'samosa', 'sashimi', 'scallops', 'seaweed_salad', 'shrimp_and_grits',
    'spaghetti_bolognese', 'spaghetti_carbonara', 'spring_rolls', 'steak',
    'strawberry_shortcake', 'sushi', 'tacos', 'takoyaki', 'tiramisu',
    'tuna_tartare', 'waffles'
]
"""

# Save the YAML file
yaml_path = "/content/Food101_YOLO/data.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"✅ data.yaml file created at {yaml_path}")


✅ data.yaml file created at /content/Food101_YOLO/data.yaml


In [ ]:
!cat /content/Food101_YOLO/data.yaml


train: /content/Food101_YOLO/images/train
val: /content/Food101_YOLO/images/val
test: /content/Food101_YOLO/images/test

nc: 101  # Number of classes
names: [
    'apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare',
    'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito',
    'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake',
    'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla',
    'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder',
    'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes',
    'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict',
    'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras',
    'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari',
    'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad',
    'grilled_cheese_sandwich', 'grilled_salmon', 'guaca

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 Nano model
model = YOLO("yolov8n.pt")  # Using the pre-trained YOLOv8 Nano model

# Train the model
# Model will be change based on the result
model.train(
    data="/content/Food101_YOLO/data.yaml",  # Path to dataset config
    epochs=10,  # Increase if you want better accuracy
    batch=32,  # Batch size
    imgsz=256,  # Image size (256x256)
    device="cuda"  # Use GPU for training
)


Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Food101_YOLO/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=256, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning /content/Food101_YOLO/labels/train... 19439 images, 3864 backgrounds, 0 corrupt:  24%|██▍       | 23303/96621 [01:00<03:03, 398.52it/s]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))
train: Scanning /content/Food101_YOLO/labels/train... 80799 images, 15821 backgrounds, 1 corrupt: 100%|██████████| 96621/96621 [03:41<00:00, 437.17it/s]

train: WARNING ⚠️ /content/Food101_YOLO/images/train/2507710.jpg: ignoring corrupt image/label: cannot identify image file '/content/Food101_YOLO/images/train/2507710.jpg'


train: New cache created: /content/Food101_YOLO/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/Food101_YOLO/labels/val... 2499 images, 2135 backgrounds, 0 corrupt:  25%|██▍       | 4634/18852 [00:12<00:31, 450.43it/s]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))
val: Scanning /content/Food101_YOLO/labels/val... 10100 images, 8752 backgrounds, 0 corrupt: 100%|██████████| 18852/18852 [00:36<00:00, 517.99it/s]


...

      10/10      1.59G    0.02826      1.341     0.8775         11        256: 100%|██████████| 3020/3020 [11:49<00:00,  4.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 295/295 [02:00<00:00,  2.46it/s]


                   all      18852      10100      0.308      0.558       0.32      0.272

10 epochs completed in 1.991 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.9MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.9MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,341,027 parameters, 0 gradients, 9.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 295/295 [01:57<00:00,  2.52it/s]


                   all      18852      10100      0.307      0.557       0.32      0.272
             apple_pie         96         96      0.126     0.0521     0.0898     0.0603
        baby_back_ribs        116        116      0.312       0.69      0.341      0.293
               baklava        117        117      0.306      0.675      0.338      0.271
        beef_carpaccio        113        113      0.353      0.805      0.475      0.441
          beef_tartare         94         94      0.277      0.564      0.313      0.255
            beet_salad        108        108      0.313       0.37      0.292      0.236
              beignets         95         95      0.374      0.842      0.466      0.436
              bibimbap        100        100      0.377       0.78      0.447      0.401
         bread_pudding        113        113      0.307      0.259       0.25      0.188
     breakfast_burrito        112        112      0.448      0.179      0.218        0.2
            bruschett

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3c7270b250>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,